# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#List-Exogenous-SymPy-Variables"><span class="toc-item-num">3 - </span>List Exogenous <code>SymPy</code> Variables</a></div><div class="lev1"><a href="#Structure-of-Model-for-Valuation-(conditional-upon-survival)"><span class="toc-item-num">4 - </span>Structure of Model for Valuation <em>(conditional upon survival)</em></a></div><div class="lev1"><a href="#Calculate-Actual-Values"><span class="toc-item-num">5 - </span>Calculate Actual Values</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas
from pprint import pprint
import sympy

from FinSymPy.Valuation import \
    terminal_value as tv, \
    present_value as pv, \
    net_present_value as npv
    
from HelpyFuncs.SymPy import sympy_eval_by_theano

Using gpu device 0: GeForce GT 750M


# Set _CONSTANTS_

In [2]:
# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .4

# Risk-Free Rate
RISK_FREE_RATE = .03

# Public Market Return
PUB_MKT_RETURN = .11

# VC fee premium
VC_FEE_PREM = .02   # .07

# CAPM Betas
PUB_MKT_BETA = 1.
PEVC_BETA_FOR_PRO_FORMA_PERIOD = 2.

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .04

# Year 0
YEAR_0 = 2001

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 4
FINAL_PRO_FORMA_YEAR = YEAR_0 + NB_PRO_FORMA_YEARS

# List Exogenous `SymPy` Variables

In [3]:
# Cash Flow components
EBIT = sympy.symbols(
    'EBIT_%d:%d' % (YEAR_0, FINAL_PRO_FORMA_YEAR + 1))
print('EBIT variables:\n', EBIT)

Depn = sympy.symbols(
    'Depn_%d:%d' % (YEAR_0, FINAL_PRO_FORMA_YEAR + 1))
print('\nDepreciation variables:\n', Depn)


# Corporate Tax Rate
corp_tax_rate = sympy.Symbol('corp_tax_rate')
print('\nCorporate Tax Rate variable:\n', corp_tax_rate)


# Long-Term Growth Rate
long_term_growth_rate = sympy.Symbol('long_term_growth_rate')
print('\nLong-Term Growth Rate variable:\n', long_term_growth_rate)


# CAPM Paramaters
risk_free_rate = sympy.Symbol('risk_free_rate')
print('\nRisk-Free Rate variable:\n', risk_free_rate)

pub_mkt_return = sympy.Symbol('pub_mkt_return')
print('\nPublic Market Return variable:\n', pub_mkt_return)

vc_fee_prem = sympy.Symbol('vc_fee_prem')
print('\nVenture Capital Fee Premium variable:\n', vc_fee_prem)

stabilized_beta = sympy.Symbol('stabilized_beta')
print('\nStabilized Beta variable:\n', stabilized_beta)

pevc_beta = sympy.Symbol('pevc_beta')
print('\nPrivate Equity / Venture Capital Beta variable:\n', pevc_beta)


# assemble all exogenous variables into a collection
exo_vars = \
    EBIT + Depn + \
    (corp_tax_rate,
     long_term_growth_rate,
     vc_fee_prem,
     risk_free_rate, pub_mkt_return,
     stabilized_beta, pevc_beta)

EBIT variables:
 (EBIT_2001, EBIT_2002, EBIT_2003, EBIT_2004, EBIT_2005)

Depreciation variables:
 (Depn_2001, Depn_2002, Depn_2003, Depn_2004, Depn_2005)

Corporate Tax Rate variable:
 corp_tax_rate

Long-Term Growth Rate variable:
 long_term_growth_rate

Risk-Free Rate variable:
 risk_free_rate

Public Market Return variable:
 pub_mkt_return

Venture Capital Fee Premium variable:
 vc_fee_prem

Stabilized Beta variable:
 stabilized_beta

Private Equity / Venture Capital Beta variable:
 pevc_beta


# Structure of Model for Valuation _(conditional upon survival)_

EBIAT:

In [4]:
EBIAT = map(
    lambda x: x - corp_tax_rate * sympy.Max(x, 0),
    EBIT)

print('EBIAT =')
pprint(EBIAT)

EBIAT =
[EBIT_2001 - corp_tax_rate*Max(0, EBIT_2001),
 EBIT_2002 - corp_tax_rate*Max(0, EBIT_2002),
 EBIT_2003 - corp_tax_rate*Max(0, EBIT_2003),
 EBIT_2004 - corp_tax_rate*Max(0, EBIT_2004),
 EBIT_2005 - corp_tax_rate*Max(0, EBIT_2005)]


Free Cash Flows before Terminal Value equal EBIAT:

In [5]:
FCF_before_TV = map(
    lambda x: reduce(lambda u, v: u + v, x),
    zip(EBIAT, Depn))

print('Free Cash Flows before Terminal Value =')
pprint(FCF_before_TV)

Free Cash Flows before Terminal Value =
[Depn_2001 + EBIT_2001 - corp_tax_rate*Max(0, EBIT_2001),
 Depn_2002 + EBIT_2002 - corp_tax_rate*Max(0, EBIT_2002),
 Depn_2003 + EBIT_2003 - corp_tax_rate*Max(0, EBIT_2003),
 Depn_2004 + EBIT_2004 - corp_tax_rate*Max(0, EBIT_2004),
 Depn_2005 + EBIT_2005 - corp_tax_rate*Max(0, EBIT_2005)]


Discount Rates:

In [6]:
pro_forma_period_discount_rate = \
    risk_free_rate + pevc_beta * (pub_mkt_return - risk_free_rate) + vc_fee_prem

print(
    'Pro-Forma Period Discount Rate =\n',
    pro_forma_period_discount_rate)

Pro-Forma Period Discount Rate =
 pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + vc_fee_prem


In [7]:
long_term_discount_rate = \
    risk_free_rate + stabilized_beta * (pub_mkt_return - risk_free_rate)
    
print(
    'Long-Term Discount Rate =\n',
    long_term_discount_rate)

Long-Term Discount Rate =
 risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate)


Terminal Value:

In [8]:
TV = tv(FCF_before_TV[-1], long_term_discount_rate, long_term_growth_rate)

print('Terminal Value =\n', TV)

Terminal Value =
 (long_term_growth_rate + 1)*(Depn_2005 + EBIT_2005 - corp_tax_rate*Max(0, EBIT_2005))/(-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))


Valuation:

In [9]:
val_of_FCF_before_TV = npv([0] + FCF_before_TV[1:], pro_forma_period_discount_rate)

val_of_TV = pv(TV, long_term_discount_rate, NB_PRO_FORMA_YEARS)

val = val_of_FCF_before_TV + val_of_TV

print('Valuation =\n', val)

Valuation =
 (long_term_growth_rate + 1)*(Depn_2005 + EBIT_2005 - corp_tax_rate*Max(0, EBIT_2005))/((-long_term_growth_rate + risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate))*(risk_free_rate + stabilized_beta*(pub_mkt_return - risk_free_rate) + 1)**4) + (Depn_2002 + EBIT_2002 - corp_tax_rate*Max(0, EBIT_2002))/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + vc_fee_prem + 1) + (Depn_2003 + EBIT_2003 - corp_tax_rate*Max(0, EBIT_2003))/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + vc_fee_prem + 1)**2 + (Depn_2004 + EBIT_2004 - corp_tax_rate*Max(0, EBIT_2004))/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + vc_fee_prem + 1)**3 + (Depn_2005 + EBIT_2005 - corp_tax_rate*Max(0, EBIT_2005))/(pevc_beta*(pub_mkt_return - risk_free_rate) + risk_free_rate + vc_fee_prem + 1)**4


# Calculate Actual Values

Import data:

In [10]:
ultra_visual_data_df = \
    pandas.read_csv(
        'data/UltraVisual.csv',
        index_col='Year',
        dtype=float)

pandas.options.display.float_format = '{:,.0f}'.format
ultra_visual_data_df

,Revenue,COGS,Gross Margin,G&A,Marketing_Sales_Expenses,Operating_Income,Depreciation,EBIT,Taxes,Net_Income
Year,,,,,,,,,,
"2,001",0,0,0,"1,862,248","184,815","-2,047,063","22,615","-2,069,678",0,"-2,069,678"
"2,002","3,541,750","345,000","3,196,750","4,132,783","1,278,700","-2,214,733","50,563","-2,265,296",0,"-2,265,296"
"2,003","20,430,250","1,895,000","18,535,250","6,472,525","7,414,100","4,648,625","131,250","4,517,375","1,806,950","2,710,425"
"2,004","39,059,250","3,525,000","35,534,250","9,708,788","14,213,700","11,611,763","221,325","11,390,438","4,556,175","6,834,263"
"2,005","65,883,000","5,890,000","59,993,000","14,563,181","23,997,200","21,432,619","324,911","21,107,708","8,443,083","12,664,625"


Exogenous Variables' values:

In [11]:
exo_vars_values_dict = \
    dict(
        corp_tax_rate=US_CORP_TAX_RATE,
        risk_free_rate=RISK_FREE_RATE,
        pub_mkt_return=PUB_MKT_RETURN,
        vc_fee_prem=VC_FEE_PREM,
        pevc_beta=PEVC_BETA_FOR_PRO_FORMA_PERIOD,
        stabilized_beta=PUB_MKT_BETA,
        long_term_growth_rate=LONG_TERM_GROWTH_RATE)
    
exo_vars_values_dict.update(
    dict(
        zip([i.name for i in EBIT],
            ultra_visual_data_df.EBIT.tolist()) +
        zip([i.name for i in Depn],
            ultra_visual_data_df.Depreciation.tolist())))

pprint(exo_vars_values_dict)

{'Depn_2001': 22615.0,
 'Depn_2002': 50563.0,
 'Depn_2003': 131250.0,
 'Depn_2004': 221325.0,
 'Depn_2005': 324911.0,
 'EBIT_2001': -2069678.0,
 'EBIT_2002': -2265296.0,
 'EBIT_2003': 4517375.0,
 'EBIT_2004': 11390438.0,
 'EBIT_2005': 21107708.0,
 'corp_tax_rate': 0.4,
 'long_term_growth_rate': 0.04,
 'pevc_beta': 2.0,
 'pub_mkt_return': 0.11,
 'risk_free_rate': 0.03,
 'stabilized_beta': 1.0,
 'vc_fee_prem': 0.02}


Function to calculate values from `SymPy` expression:

In [12]:
def calc(x):
    if isinstance(x, (list, tuple)):
        return [float(calc(i)) for i in x]
    else:
        return float(sympy_eval_by_theano(x, symbols=exo_vars, **exo_vars_values_dict))

Valuation Calculations:

In [13]:
print('VALUATION CONDITIONAL UPON SURVIVAL')
print('___________________________________\n')

print('U.S. Corporate Tax Rate =   %s\n' % '{:.0%}'.format(calc(corp_tax_rate)))

print('Risk-Free Rate =   %s' % '{:.0%}'.format(calc(risk_free_rate)))
print('Public-Market Return =   %s\n' % '{:.0%}'.format(calc(pub_mkt_return)))
print('VC Fee Premium =   %s\n' % '{:.0%}'.format(calc(vc_fee_prem)))

print('Stabilized Beta =   %.0fx' % calc(stabilized_beta))
print('Long-Term Discount Rate =   %s\n' % '{:.0%}'.format(calc(long_term_discount_rate)))

print('PEVC Beta =   %.0fx' % calc(pevc_beta))
print('Discount Rate for Pro-Forma Period =   %s\n' % '{:.0%}'.format(calc(pro_forma_period_discount_rate)))

print('Valuation of FCF before TV =   $%.2f MM' % (calc(val_of_FCF_before_TV) / 1e6))
print('Valuation of TV =   $%.2f MM' % (calc(val_of_TV) / 1e6))
print('\nVALUATION | survival =   $%.2f MM\n' % (calc(val) / 1e6))

print('Detailed Cash Flow Calcs:')
val_calcs_df = pandas.DataFrame(index=['Year 0'] + range(YEAR_0 + 1, FINAL_PRO_FORMA_YEAR + 1))
# ref: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html
pandas.options.display.float_format = '{:,.0f}'.format

val_calcs_df['EBIT'] = calc(EBIT)
val_calcs_df['EBIAT'] = calc(EBIAT)
val_calcs_df['Depreciation'] = calc(Depn)
val_calcs_df['FCF before Terminal Value'] = calc(FCF_before_TV)
val_calcs_df['Terminal Value'] = numpy.nan
val_calcs_df.loc[FINAL_PRO_FORMA_YEAR, 'Terminal Value'] = calc(TV)

val_calcs_df.T

VALUATION CONDITIONAL UPON SURVIVAL
___________________________________

U.S. Corporate Tax Rate =   40%

Risk-Free Rate =   3%
Public-Market Return =   11%

VC Fee Premium =   2%

Stabilized Beta =   1x
Long-Term Discount Rate =   11%

PEVC Beta =   2x
Discount Rate for Pro-Forma Period =   21%

Valuation of FCF before TV =   $10.15 MM
Valuation of TV =   $127.13 MM

VALUATION | survival =   $137.28 MM

Detailed Cash Flow Calcs:


,Year 0,2002,2003,2004,2005
EBIT,"-2,069,678","-2,265,296","4,517,375","11,390,438","21,107,708"
EBIAT,"-2,069,678","-2,265,296","2,710,425","6,834,262","12,664,625"
Depreciation,"22,615","50,563","131,250","221,325","324,911"
FCF before Terminal Value,"-2,047,063","-2,214,733","2,841,675","7,055,588","12,989,537"
Terminal Value,nan,nan,nan,nan,"192,987,408"
